In [26]:
import json
from src.datasets.simple_dataset import SimpleDataset
import torch

In [27]:
data_dir = "../data/Train_Data"
dataset = SimpleDataset(data_dir=data_dir)

In [28]:
dataset[3]["profiled_gen_utilization"]

tensor([[1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.]])

In [29]:
ut = dataset[3]["profiled_gen_utilization"]
indices = torch.where(ut != 1.0)
indices

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
         28, 29, 30, 31, 42, 43, 44, 45, 46, 47, 48, 49, 49, 49, 49, 49, 49, 50,
         50, 50, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
         51, 51, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
         52, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
         54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 55,
         55, 55, 55, 55, 55, 66, 67, 68, 69, 70, 71]),
 tensor([15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,  0,  7,  9, 10, 13, 15,  1,
          2,  6,  7,  8, 10, 11, 14, 15,  1,  3,  5,  7,  8,  9, 10, 11, 13, 14,
         15, 16,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
          0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 1

In [30]:
ut[51, 0]

tensor(1.)

In [31]:
eps = 1e-4
util = dataset[2]["profiled_gen_utilization"]
mask = (util - 1.0).abs().gt(eps) | torch.isnan(util) | torch.isinf(util)
t_idx, g_idx = torch.where(mask)

# print a few offenders with full precision
for k in range(t_idx.numel()):
    t = t_idx[k].item()
    g = g_idx[k].item()
    val = util[t, g].item()
    print(f"(t={t}, g={g}) util={val:.18f}  diff={abs(val-1.0):.3e}")

(t=0, g=15) util=0.000000000000000000  diff=1.000e+00
(t=1, g=15) util=0.000000000000000000  diff=1.000e+00
(t=2, g=15) util=0.000000000000000000  diff=1.000e+00
(t=3, g=15) util=0.000000000000000000  diff=1.000e+00
(t=4, g=15) util=0.000000000000000000  diff=1.000e+00
(t=5, g=15) util=0.000000000000000000  diff=1.000e+00
(t=6, g=15) util=0.000000000000000000  diff=1.000e+00
(t=7, g=15) util=0.000000000000000000  diff=1.000e+00
(t=18, g=15) util=0.000000000000000000  diff=1.000e+00
(t=19, g=15) util=0.000000000000000000  diff=1.000e+00
(t=20, g=15) util=0.000000000000000000  diff=1.000e+00
(t=21, g=15) util=0.000000000000000000  diff=1.000e+00
(t=22, g=15) util=0.000000000000000000  diff=1.000e+00
(t=23, g=15) util=0.000000000000000000  diff=1.000e+00
(t=24, g=15) util=0.000000000000000000  diff=1.000e+00
(t=25, g=0) util=0.000000000000000000  diff=1.000e+00
(t=25, g=4) util=0.837737798690795898  diff=1.623e-01
(t=25, g=9) util=0.000000000000000000  diff=1.000e+00
(t=25, g=10) util=0.0

In [32]:
util[1, 16]

tensor(1.)

In [33]:
gens_not_fully_utilized = []
for i in range(len(dataset)):
    util = dataset[i]["profiled_gen_utilization"]
    is_zero = util.abs() <= eps
    min_util_that_is_non_zero = util[~is_zero].min().item() if (~is_zero).any() else float('inf')
    is_one  = (util - 1.0).abs() <= eps

    bad_mask = ~(is_zero | is_one) | torch.isnan(util) | torch.isinf(util)
    gen_mask = bad_mask.any(dim=0)
    gen_indices = torch.where(gen_mask)[0].unique()
    print(f"Sample {i}: Non-unity or non-zero utilization at indices {gen_indices}, min non-zero utilization: {min_util_that_is_non_zero}")
    gens_not_fully_utilized.append(gen_indices)

Sample 0: Non-unity or non-zero utilization at indices tensor([], dtype=torch.int64), min non-zero utilization: 1.0
Sample 1: Non-unity or non-zero utilization at indices tensor([], dtype=torch.int64), min non-zero utilization: 1.0
Sample 2: Non-unity or non-zero utilization at indices tensor([ 1,  3,  4,  6,  7,  9, 11, 12, 13, 14, 16]), min non-zero utilization: 0.051858000457286835
Sample 3: Non-unity or non-zero utilization at indices tensor([ 7, 13, 14, 16]), min non-zero utilization: 0.11516500264406204
Sample 4: Non-unity or non-zero utilization at indices tensor([ 6,  7,  8,  9, 14, 16]), min non-zero utilization: 0.6537399888038635
Sample 5: Non-unity or non-zero utilization at indices tensor([], dtype=torch.int64), min non-zero utilization: 0.9999998807907104
Sample 6: Non-unity or non-zero utilization at indices tensor([ 3, 11, 16]), min non-zero utilization: 0.1144462525844574
Sample 7: Non-unity or non-zero utilization at indices tensor([], dtype=torch.int64), min non-zero